In [1]:
import os
import pandas as pd
import requests
import pathlib
from requests.auth import HTTPBasicAuth

api_key = os.environ.get("NCBI_API_KEY")


In [2]:
# Input
taxon_file = "../data/imported/taxids.list"

# Output
output_dir = pathlib.Path("../data/generated/download_genomes")
pathlib.Path.mkdir(output_dir, exist_ok=True)
genomes_metadata_file = output_dir / "genomes_metadata.csv"
genome_accessions_file = output_dir / "genome_accessions.list"
genomes_zip_file = output_dir / "genomes.zip"


In [3]:
def get_genome_dataset_report(taxon_id, api_key, params=None):
    """
    Get the genome dataset report for a given taxon id using NCBI API.
    Default parameters are:
    - reference_only=true
    - assembly_source=refseq
    - exclude_atypical=true # => don't include genomes that have assembly issues or are otherwise atypical
    - assembly_version=current
    """
    if params is None:
        params = {
            "filters.reference_only": "true",
            "filters.assembly_source": "refseq",
            "filters.exclude_atypical": "true",
            "filters.assembly_version": "current",
        }
    auth = HTTPBasicAuth("api-key", api_key)
    url = f"https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/taxon/{taxon_id}/dataset_report"
    response = requests.get(url=url, auth=auth, params=params)
    if "reports" not in response.json():
        print(f"No reports found for taxon id {taxon_id}")
        return
    return pd.DataFrame(
        _parse_genome_dataset_report_response(response.json(), taxon_id)
    )


def _parse_genome_dataset_report_response(response, query_taxid):
    for report in response["reports"]:
        yield {
            "query_taxid": query_taxid,
            "genome_accession": report["current_accession"],
            "genome_name": report["organism"]["organism_name"],
            "genome_taxid": report["organism"]["tax_id"],
            "assembly_level": report["assembly_info"]["assembly_level"],
            "assembly_status": report["assembly_info"]["assembly_status"],
            "assembly_name": report["assembly_info"]["assembly_name"],
            "refseq_category": report["assembly_info"]["refseq_category"]
            if "refseq_category" in report["assembly_info"]
            else pd.NA,
            "number_of_contigs": report["assembly_stats"]["number_of_contigs"],
            "contig_n50": report["assembly_stats"]["contig_n50"],
            "contig_l50": report["assembly_stats"]["contig_l50"],
            "genome_coverage": float(
                str(report["assembly_stats"]["genome_coverage"]).replace("x", "")
            )
            if "genome_coverage" in report["assembly_stats"]
            else pd.NA,
            "total_sequence_length": report["assembly_stats"]["total_sequence_length"],
            "total_ungapped_length": report["assembly_stats"]["total_ungapped_length"],
        }


In [4]:
refseq_params = {
    "filters.reference_only": "true",
    "filters.assembly_source": "refseq",
    "filters.exclude_atypical": "true",
    "filters.assembly_version": "current",
}

genome_dataset_report = get_genome_dataset_report("562", api_key, params=refseq_params)
genome_dataset_report


,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,model,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length
0,562,GCF_000005845.2,Escherichia coli str. K-12 substr. MG1655,511145,Complete Genome,current,ASM584v2,reference genome,<NA>,1,4641652,1,<NA>,4641652,4641652
1,562,GCF_000008865.2,Escherichia coli O157:H7 str. Sakai,386585,Complete Genome,current,ASM886v2,reference genome,<NA>,3,5498578,1,<NA>,5594605,5594605


In [5]:
def read_taxon_list(taxon_file):
    """
    Read a list of taxon ids from a file.
    """
    taxon_list = []
    with open(taxon_file) as f:
        taxon_list.extend(line.strip() for line in f if not line.startswith("#"))
    return list(set(taxon_list))


taxon_list = read_taxon_list(taxon_file)
print(f"{len(taxon_list)} taxon ids read from {taxon_file}")


141 taxon ids read from ../data/imported/taxids.list


In [6]:
refseq_genomes_dataset_reports = [
    get_genome_dataset_report(taxon_id, api_key, params=refseq_params)
    for taxon_id in taxon_list
]
refseq_genomes_df = pd.concat(refseq_genomes_dataset_reports).reset_index(drop=True)
refseq_genomes_df.sample()


No reports found for taxon id 1955243
No reports found for taxon id 39491
No reports found for taxon id 384638
No reports found for taxon id 2053618
No reports found for taxon id 1506
No reports found for taxon id 29523
No reports found for taxon id 84026
No reports found for taxon id 1945593
No reports found for taxon id 1682
No reports found for taxon id 76856
No reports found for taxon id 1535
No reports found for taxon id 39492
No reports found for taxon id 1891238
No reports found for taxon id 1869337
No reports found for taxon id 1898207
No reports found for taxon id 1872530
No reports found for taxon id 2023260


,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,model,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length
74,544645,GCF_016889065.1,Butyricimonas virosa,544645,Complete Genome,current,ASM1688906v1,representative genome,<NA>,1,4813143,1,611.9,4813143,4813143


In [7]:
refseq_genomes_df["refseq_category"].value_counts()


representative genome    122
reference genome           3
Name: refseq_category, dtype: int64

In [8]:
refseq_genomes_df["assembly_level"].value_counts()


Complete Genome    68
Scaffold           35
Contig             14
Chromosome          8
Name: assembly_level, dtype: int64

In [9]:
failed_taxon_ids = list(
    set(taxon_list).difference(set(refseq_genomes_df["query_taxid"]))
)
print(f"{len(failed_taxon_ids)} taxon ids failed to retrieve genome dataset report")
failed_taxon_ids


17 taxon ids failed to retrieve genome dataset report


['384638',
 '2053618',
 '1869337',
 '39491',
 '84026',
 '1682',
 '29523',
 '39492',
 '1898207',
 '1872530',
 '76856',
 '1891238',
 '2023260',
 '1955243',
 '1535',
 '1945593',
 '1506']

In [10]:
genbank_params = {
    "filters.assembly_source": "genbank",
    "filters.exclude_atypical": "true",
    "filters.assembly_version": "current",
    "table_fields": "assminfo-accession,assminfo-name",
    "page_size": 500,
}

genbank_genome_dataset_reports = [
    get_genome_dataset_report(failed_taxon_id, api_key, params=genbank_params)
    for failed_taxon_id in failed_taxon_ids
]
genbank_genomes_df = pd.concat(genbank_genome_dataset_reports).reset_index(drop=True)
genbank_genomes_df.sample(5)


,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,model,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length
1924,1955243,GCA_017459635.1,Blautia sp.,1955243,Contig,current,ASM1745963v1,<NA>,<NA>,109,40166,22,30.0,2669729,2669729
1095,1898207,GCA_004558435.1,Clostridiales bacterium,1898207,Contig,current,ASM455843v1,<NA>,<NA>,154,17005,31,32.4,1698269,1698269
1797,1891238,GCA_026417035.1,Burkholderiales bacterium,1891238,Scaffold,current,ASM2641703v1,<NA>,<NA>,286,42790,27,13.6,3770068,3769698
878,1898207,GCA_003451875.1,Clostridiales bacterium,1898207,Scaffold,current,ASM345187v1,<NA>,<NA>,366,5472,82,92.5,1503345,1485905
1620,1891238,GCA_019635025.1,Burkholderiales bacterium,1891238,Contig,current,ASM1963502v1,<NA>,<NA>,156,30954,26,14.0,2752864,2752864


In [11]:
genbank_genomes_df["assembly_level"].value_counts()


Series([], Name: model, dtype: int64)

In [12]:
def select_best_representative(df, consider_coverage=None):
    """
    Select the best representative genome from a genome dataset report.
    """
    if consider_coverage is None:
        consider_coverage = True
    taxa_dfs = []
    # convert columns to correct data types
    df = df.astype(
        {
            "query_taxid": "int",
            "genome_taxid": "int",
            "number_of_contigs": "int",
            "total_sequence_length": "int",
            "total_ungapped_length": "int",
            "contig_n50": "int",
            "contig_l50": "int",
            # "genome_coverage": "float",
        }
    )
    df["gaps"] = df["total_sequence_length"] - df["total_ungapped_length"]
    df["bases_per_contig"] = df["total_ungapped_length"] / df["number_of_contigs"]

    if consider_coverage:
        # if genome coverage is NA remove it from the dataframe
        df = df.dropna(subset=["genome_coverage"]).copy()

    # Select the representative genome with the highest genome coverage, least gaps and least contig_l50 and highest contig_n50.
    for _, grouped_taxa_df in df.groupby("query_taxid"):
        rows = grouped_taxa_df.shape[0]
        if rows == 1:
            taxa_dfs.append(grouped_taxa_df.iloc[0])
            continue

        # if a taxon has any 'Complete Genome's then shortlist those genomes to be considered for a representative genome.
        if any(grouped_taxa_df["assembly_level"] == "Complete Genome"):
            grouped_taxa_df = grouped_taxa_df.query(
                "assembly_level == 'Complete Genome'"
            ).copy()

        # Select the representative genome with the highest genome coverage, least gaps and least contig_l50 and highest contig_n50.
        grouped_taxa_df = grouped_taxa_df.sort_values(
            ["bases_per_contig", "contig_n50", "contig_l50", "genome_coverage", "gaps"],
            ascending=[False, False, True, False, True],
        )

        # append the best representative genome to the taxa_dfs list
        taxa_dfs.append(grouped_taxa_df.iloc[0])

    return pd.DataFrame(taxa_dfs)


select_best_representative(genbank_genomes_df).reset_index(drop=True).sample()


,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length,gaps,bases_per_contig
4,39491,GCA_022453685.1,[Eubacterium] rectale,39491,Complete Genome,current,ASM2245368v1,<NA>,1,3451485,1,129.4,3451485,3451485,0,3451485.0


In [13]:
genomes_df = pd.concat(
    [
        select_best_representative(refseq_genomes_df, consider_coverage=False),
        select_best_representative(genbank_genomes_df),
    ]
).reset_index(drop=True)
genomes_df.to_csv(genomes_metadata_file, index=False)
genomes_df.sample(5)


,query_taxid,genome_accession,genome_name,genome_taxid,assembly_level,assembly_status,assembly_name,refseq_category,number_of_contigs,contig_n50,contig_l50,genome_coverage,total_sequence_length,total_ungapped_length,gaps,bases_per_contig
39,28127,GCF_020735565.1,Hoylesella buccalis,28127,Complete Genome,current,ASM2073556v1,representative genome,1,3308411,1,7609.1,3308411,3308411,0,3.308411e+06
120,691816,GCF_000614125.1,Bacteroides rodentium JCM 16496,1236512,Contig,current,ASM61412v1,representative genome,148,78229,19,27.0,4884492,4884492,0,3.300332e+04
102,310297,GCF_902374375.1,Phocaeicola plebeius,310297,Scaffold,current,MGYG-HGUT-01364,representative genome,25,478534,4,10.0,4421924,4421324,600,1.768530e+05
12,1264,GCF_000179635.2,Ruminococcus albus 7 = DSM 20455,697329,Complete Genome,current,ASM17963v2,representative genome,5,3685408,1,30.0,4482087,4482087,0,8.964174e+05
56,40520,GCF_003468995.1,Blautia obeum,40520,Scaffold,current,ASM346899v1,representative genome,35,320905,5,100.0,3889284,3889204,80,1.111201e+05


In [14]:
## Check to make sure there are no duplicate genome accessions
assert all(
    genomes_df["genome_accession"].value_counts() == 1
), "Duplicate genome accessions found!"


In [15]:
# Save the genome accessions to a file
genomes_df["genome_accession"].to_csv(genome_accessions_file, index=False, header=False)


## Download the genomes

In [16]:
%%bash -s "$api_key" "$genome_accessions_file" "$genomes_zip_file"

datasets download genome accession \
    --api-key $1 \
    --inputfile $2 \
    --no-progressbar \
    --filename $3
    